In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#additional imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
#data preprocessing

# Apply age categorization transformation to the combined dataset
age_categorization = lambda age: -1 if pd.isnull(age) \
                                  else 0 if age <= 15 \
                                  else 1 if age <= 32 \
                                  else 2 if age <= 48 \
                                  else 3 if age <= 64 \
                                  else 4

# Apply the age categorization to both training and test datasets
train_data['Age_Group'] = train_data['Age'].apply(age_categorization)
test_data['Age_Group'] = test_data['Age'].apply(age_categorization)

# Drop the original 'Age' column from both datasets
train_data.drop('Age', axis=1, inplace=True)
test_data.drop('Age', axis=1, inplace=True)

#is_alone
train_data['alone'] = np.where((train_data['SibSp'] + train_data['Parch']) == 0, 0, 1)
test_data['alone'] = np.where((test_data['SibSp'] + test_data['Parch']) == 0, 0, 1)

# Drop the 'SibSp' and 'Parch' columns from both datasets
train_data.drop(['SibSp', 'Parch'], axis=1, inplace=True)
test_data.drop(['SibSp', 'Parch'], axis=1, inplace=True)


# Now both datasets are processed separately but consistently
# Display the first few rows of both datasets to verify
print("Processed Training Data Preview:")
print(train_data.head())
print("\nProcessed Test Data Preview:")
print(test_data.head())


In [ ]:



y = train_data["Survived"]

features = ["Pclass", "Sex","Ticket", "Fare","Embarked","Age_Group", "alone"]


#catigorical 
catigorical = ["Sex", "Embarked", "Pclass"]
#numerical
numerical = ["Age", "Fare", "SibSp", "Parch"]



X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),  # Handle missing values
    ('std_scaler', StandardScaler()),  # Scale the features
    ("model", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1))  # Use Logistic Regression
])

X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)
pipeline.fit(X, y)

#model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
#model.fit(X, y)
predictions = pipeline.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")